# GoodFoods — Full Reservation Agent (Final)
Run this notebook in Google Colab. It creates code files, data files, and launches Streamlit with a tunnel. If GROK_API_KEY is set, Grok will be used; otherwise a mock parser is used.

In [1]:

!pip install -q streamlit pyngrok pandas numpy matplotlib seaborn requests
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 22.7 MB/s eta 0:00:00


In [2]:

%%writefile generate_data.py
import json, random
cuisines=["Italian","Indian","Chinese","Mexican","Mediterranean","Japanese","Thai","French","American","Vegan"]
locations=["Koramangala","Indiranagar","MG Road","Brigade Road","Jayanagar","Whitefield","Hebbal","Rajajinagar","HSR Layout","Bannerghatta"]
price_tiers=["budget","mid","premium"]
def randcoord(): return {"lat":12.97+random.uniform(-0.05,0.05),"lon":77.59+random.uniform(-0.05,0.05)}
def gen(i):
    c=random.choice(cuisines); l=random.choice(locations); cap=random.choice([2,4,6,8,10,12,20,30])
    return {"id":i,"name":f"{l} {c} Place {i}","cuisine":c,"location":l,"capacity":cap,"price_tier":random.choice(price_tiers),"rating":round(random.uniform(3.2,4.9),1),"coords":randcoord(),"description":f"{c} cuisine at {l} seating {cap}."}
data=[gen(i) for i in range(1,101)]
json.dump(data,open("restaurants.json","w"),indent=2)
print("restaurants.json created with", len(data), "items")


Writing generate_data.py


In [3]:

%%writefile create_slots.py
import json
slots={}
for i in range(1,101):
    slots[str(i)]=["18:00","19:00","20:00","21:00"]
json.dump(slots, open("restaurant_slots.json","w"), indent=2)
print("restaurant_slots.json created")


Writing create_slots.py


In [4]:

%%writefile tools.py
import json, os, datetime
DATA_FILE="restaurants.json"
BOOKINGS_FILE="bookings.json"
SLOTS_FILE="restaurant_slots.json"
def load_restaurants():
    with open(DATA_FILE,"r") as f: return json.load(f)
def load_bookings():
    if not os.path.exists(BOOKINGS_FILE): return []
    with open(BOOKINGS_FILE,"r") as f: return json.load(f)
def save_bookings(b):
    with open(BOOKINGS_FILE,"w") as f: json.dump(b,f,indent=2,default=str)
def load_slots():
    if not os.path.exists(SLOTS_FILE): return {}
    with open(SLOTS_FILE,"r") as f: return json.load(f)
def save_slots(s):
    with open(SLOTS_FILE,"w") as f: json.dump(s,f,indent=2)
def get_available_slots(restaurant_id):
    slots=load_slots()
    return slots.get(str(restaurant_id),[])
def search_restaurants(cuisine=None,location=None,party_size=None,top_k=6):
    rests=load_restaurants()
    def score(r):
        s=0
        if cuisine and r.get("cuisine","").lower()==cuisine.lower(): s+=40
        if location and r.get("location","").lower()==location.lower(): s+=30
        if party_size and r.get("capacity",0)>=party_size: s+=20
        s+=int((r.get("rating",3.5)-3.0)*10)
        return s
    return sorted(rests,key=score,reverse=True)[:top_k]
def get_availability(restaurant_id,slot=None,party_size=None):
    rests=load_restaurants()
    r=next((x for x in rests if x["id"]==restaurant_id),None)
    if not r: return {"available":False,"reason":"not_found"}
    slots=load_slots()
    avail_slots=slots.get(str(restaurant_id),[])
    if slot:
        if slot in avail_slots: return {"available":True,"slot":slot,"capacity":r["capacity"]}
        return {"available":False,"reason":"slot_unavailable","available_slots":avail_slots}
    return {"available_slots": avail_slots, "capacity": r["capacity"]}
def book_slot(restaurant_id,slot,party_size,customer_name,contact=None):
    slots=load_slots()
    rs=slots.get(str(restaurant_id),[])
    if slot not in rs:
        return {"success":False,"reason":"slot_unavailable","available_slots":rs}
    rs.remove(slot)
    slots[str(restaurant_id)]=rs
    save_slots(slots)
    bookings=load_bookings()
    booking={"id": len(bookings)+1, "restaurant_id": restaurant_id, "slot": slot, "party_size": party_size, "customer_name": customer_name, "contact": contact, "created_at": datetime.datetime.now().isoformat()}
    bookings.append(booking)
    save_bookings(bookings)
    return {"success":True,"booking":booking}
def cancel_booking(booking_id):
    b=load_bookings()
    rem=[x for x in b if x["id"]!=booking_id]
    if len(rem)==len(b): return {"success":False,"reason":"not_found"}
    save_bookings(rem)
    return {"success":True}


Writing tools.py


In [5]:

%%writefile grok_llm.py
import os, requests, json
GROK_API_KEY=os.environ.get("GROK_API_KEY")
GROK_API_URL=os.environ.get("GROK_API_URL","https://api.grok.ai/v1/generate")
def call_grok(prompt, max_tokens=512, temperature=0.2):
    if not GROK_API_KEY: return None
    headers={"Content-Type":"application/json","Authorization":f"Bearer {GROK_API_KEY}"}
    body={"prompt":prompt,"max_tokens":max_tokens,"temperature":temperature}
    try:
        r=requests.post(GROK_API_URL, headers=headers, json=body, timeout=30)
        if r.status_code==200: return r.json()
        return {"error":f"status {r.status_code}","text":r.text}
    except Exception as e:
        return {"error":str(e)}
def parse_grok_response(resp):
    if not resp: return None
    if isinstance(resp, dict) and "text" in resp: return resp["text"]
    if isinstance(resp, dict) and "output" in resp: return json.dumps(resp["output"])
    return str(resp)


Writing grok_llm.py


In [6]:

%%writefile orchestrator.py
import re, json
from grok_llm import call_grok, parse_grok_response
from tools import search_restaurants, get_availability, book_slot, get_available_slots
def mock_llm(user):
    u=user.lower()
    if "book" in u or "reserve" in u:
        m=re.search(r"(\d+)\s*(people|persons|pax|guests)?",u)
        p=int(m.group(1)) if m else 2
        mid=re.search(r"restaurant\s*(\d+)",u)
        rid=int(mid.group(1)) if mid else None
        slot=None
        m2=re.search(r"(\d{1,2}:\d{2})",u)
        if m2: slot=m2.group(1)
        return {"intent":"book","tool":"book_table","args":{"restaurant_id":rid,"slot":slot,"party_size":p,"customer_name":"Guest"}}
    if "availability" in u or "available" in u or "free" in u:
        mid=re.search(r"restaurant\s*(\d+)",u)
        rid=int(mid.group(1)) if mid else 1
        m2=re.search(r"(\d{1,2}:\d{2})",u)
        slot=m2.group(1) if m2 else None
        return {"intent":"availability","tool":"get_availability","args":{"restaurant_id":rid,"time":slot,"party_size":2}}
    cuisine=None; location=None; p=2
    for c in ["italian","indian","chinese","mexican","mediterranean","japanese","thai","french","american","vegan"]:
        if c in u: cuisine=c.title(); break
    for loc in ["koramangala","indiranagar","mg road","brigade road","jayanagar","whitefield","hebbal","rajajinagar","hsr layout","bannerghatta"]:
        if loc in u: location=loc.title(); break
    m=re.search(r"(\d+)\s*(people|persons|pax|guests)",u)
    if m: p=int(m.group(1))
    return {"intent":"search","tool":"search_restaurants","args":{"cuisine":cuisine,"location":location,"party_size":p}}
def llm_decide(user_message):
    prompt=f"""You are an intent parser. Return JSON with keys: intent ('search'|'book'|'availability'), tool (search_restaurants/get_availability/book_table), args (dict). User: {user_message}"""
    resp=call_grok(prompt)
    text=parse_grok_response(resp)
    try:
        j=json.loads(text.strip())
        return j
    except Exception:
        return mock_llm(user_message)
def orchestrate(user_message):
    decision=llm_decide(user_message)
    tool=decision.get("tool")
    args=decision.get("args",{})
    if tool=="search_restaurants":
        return {"llm_decision":decision,"tool_result": search_restaurants(cuisine=args.get("cuisine"), location=args.get("location"), party_size=args.get("party_size"))}
    if tool=="get_availability":
        return {"llm_decision":decision,"tool_result": get_availability(restaurant_id=int(args.get("restaurant_id")), slot=args.get("slot") or args.get("time"), party_size=int(args.get("party_size",2)))}
    if tool=="book_table" or tool=="book_slot":
        return {"llm_decision":decision,"tool_result": book_slot(restaurant_id=int(args.get("restaurant_id")), slot=args.get("slot") or args.get("time"), party_size=int(args.get("party_size",2)), customer_name=args.get("customer_name","Guest"), contact=args.get("contact"))}
    return {"llm_decision":decision,"tool_result":None}


Writing orchestrator.py


In [7]:

%%writefile app.py
import streamlit as st, json, pandas as pd, time
from orchestrator import orchestrate
from tools import get_available_slots
st.set_page_config(layout="wide")
if "history" not in st.session_state: st.session_state.history=[]
st.title("GoodFoods — Reservation Agent (Final)")
col1,col2=st.columns([2,1])
with col1:
    user=st.text_input("You:","Find Italian restaurants near Koramangala.")
    if st.button("Send"):
        st.session_state.history.append({"role":"user","text":user})
        out=orchestrate(user)
        tr=out.get("tool_result")
        if isinstance(tr,list):
            display=[f"[{r['id']}] {r['name']} | {r['cuisine']} | {r['location']} | cap:{r['capacity']} | rating:{r['rating']}" for r in tr]
            st.session_state.history.append({"role":"agent","text":"\n".join(display)})
        else:
            st.session_state.history.append({"role":"agent","text":str(tr)})
    st.subheader("Chat")
    for m in st.session_state.history:
        if m["role"]=="user": st.markdown(f"**You:** {m['text']}")
        else: st.markdown(f"**Agent:** {m['text']}")
    st.markdown("---")
    st.markdown("### Manual booking")
    rcol1,rcol2,rcol3=st.columns(3)
    rid=rcol1.number_input("Restaurant ID", min_value=1, max_value=100, value=1)
    slots=get_available_slots(rid)
    slot_choice=rcol2.selectbox("Slot", options=slots if slots else ["No slots available"])
    party=rcol3.number_input("Party size", min_value=1, max_value=30, value=2)
    name=st.text_input("Your name","Guest")
    if st.button("Book Slot Manually"):
        if slot_choice=="No slots available": st.error("No slot selected")
        else:
            cmd=f"book restaurant {rid} at {slot_choice} for {party} people"
            st.session_state.history.append({"role":"user","text":cmd})
            out=orchestrate(cmd)
            st.session_state.history.append({"role":"agent","text":str(out.get('tool_result'))})
with col2:
    data=json.load(open("restaurants.json"))
    df=pd.DataFrame(data)
    st.metric("Total restaurants", len(df))
    st.markdown("### Top cuisines")
    st.bar_chart(df["cuisine"].value_counts())
    st.markdown("### Rating distribution")
    st.bar_chart(df["rating"].round(1).value_counts().sort_index())
    st.markdown("### Sample restaurants table")
    st.dataframe(df.sample(10))


Writing app.py


In [8]:
!python generate_data.py

restaurants.json created with 100 items


In [9]:
!python create_slots.py

restaurant_slots.json created


In [11]:

# Kill old, start streamlit and tunnel (Cloudflare). If prefer ngrok set USE_NGROK=True and supply ngrok auth token.
USE_NGROK=False
if USE_NGROK:
    from pyngrok import ngrok
    ngrok.set_auth_token("")
    public_url = ngrok.connect(8501)
    print("ngrok tunnel:", public_url)
else:
    !pkill -9 streamlit || true
    !pkill -9 cloudflared || true
    !streamlit run app.py &>/dev/null&
    import time
    time.sleep(4)
    !./cloudflared-linux-amd64 tunnel --url http://localhost:8501


2025-11-26T19:30:57Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-26T19:30:57Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-26T19:31:02Z INF +--------------------------------------------------------------------------------------------+
2025-11-26T19:31:02Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-26T19:31:02Z INF |  https://envelope-surf-progressive-berkeley.trycloudfl